In [1]:
import os
import sys
import re
import pandas as pd
import zipfile
from enum import Enum, EnumMeta
from Tables import Tables
from Spells import Spells, CLERIC100, WIZARD100
from Items_full import Items
from SegmentTable import BaseOps, BaseTable, Segments
from IPython.display import display
from random import sample
import zlib
import shutil

In [2]:
class DataPiece(BaseOps):
    
    def __init__(self, buffer, position, length, mismatch=0):
        self.buffer = buffer
        self.position = position
        self.length = length
        self.mismatch = mismatch
        self.end = self.position + self.length
        self.signed = False
        self.order = 'little'
    
    def getBytes(self):
        return self.buffer[self.mismatch + self.position:
                           self.mismatch + self.end]
    
    def getValue(self):
        return self.bytes2Num(self.getBytes(),
                              self.signed,
                              self.order)
    
    def getString(self):
        _s = self.getBytes()
        for _i, _c in enumerate(_s):
            if _c < 32 or _c > 127:
                return str(_s[:_i], encoding='UTF-8')
        
    def setValue(self, new_value):
        if isinstance(new_value, Enum):
            new_value = new_value.value
        self.buffer[self.mismatch + self.position: 
                    self.mismatch + self.end] = self.num2Bytes(new_value,
                                                               self.length,
                                                               self.signed,
                                                               self.order)    
        
    def incValue(self, inc_value):
        self.setValue(self.getValue() + inc_value)

In [3]:
class SegmentStructure(BaseOps, Enum):
    
    def __init__(self, header_length, mismatch, 
                 offset_position, offset_length, 
                 count_position, count_length, 
                 length_position, length_length, 
                 record_length, spacing, 
                 abs_count=None, abs_length=None, extra_segment=None):
        self.header_length = header_length
        self.mismatch = mismatch
        self.offset_position = offset_position
        self.offset_length = offset_length
        self.count_position = count_position
        self.count_length = count_length
        self.length_position = length_position
        self.length_length = length_length
        self.record_length = record_length
        self.spacing = spacing
        self.abs_count = abs_count
        self.abs_length = abs_length
        self.extra_segment = extra_segment
    
    def getOffset(self, parent_buffer, parent_mismatch=0):
        if self.offset_position is None:
            return parent_mismatch
        return self.bytes2Num(parent_buffer[self.offset_position + parent_mismatch:
                                            self.offset_position + parent_mismatch + self.offset_length])
    
    def getCount(self, parent_buffer, parent_mismatch=0):
        if self.abs_count is not None:
            return self.abs_count
        elif self.count_position is not None:
            return self.bytes2Num(parent_buffer[self.count_position + parent_mismatch:
                                                self.count_position + parent_mismatch + self.count_length])
        else:
            return None
        
    def getLength(self, parent_buffer, parent_mismatch=0):
        if self.abs_length is not None:
            return self.abs_length
        elif self.length_position is not None:
            return self.bytes2Num(parent_buffer[self.length_position + parent_mismatch:
                                                self.length_position + parent_mismatch + self.length_length])
        _count = self.getCount(parent_buffer, parent_mismatch)
        if _count is not None and self.record_length is not None:
            return _count * self.record_length
        else:
            return None        
    
    def getBufferCopy(self, parent_buffer, parent_mismatch=0):
        return parent_buffer[self.getOffset(parent_buffer, 
                                            parent_mismatch) + parent_mismatch:
                             self.getOffset(parent_buffer, 
                                            parent_mismatch) + parent_mismatch + self.getLength(parent_buffer, 
                                                                                                parent_mismatch)]
    
    def updateOffset(self, parent_buffer, new_offset, parent_mismatch=0):
        if self.getLength(parent_buffer, parent_mismatch) == 0:
            new_offset = 0
        parent_buffer[self.offset_position + parent_mismatch:
                      self.offset_position + parent_mismatch + self.offset_length] = self.num2Bytes(new_offset, 
                                                                                                    self.offset_length)
    
    def updateCount(self, parent_buffer, new_count, parent_mismatch=0):
        if self.count_position is not None:
            parent_buffer[self.count_position + parent_mismatch:
                          self.count_position + parent_mismatch + self.count_length] = self.num2Bytes(new_count, 
                                                                                                      self.count_length)
            
    def incCount(self, parent_buffer, inc_count=1, parent_mismatch=0):
        self.updateCount(parent_buffer, self.getCount(parent_buffer, parent_mismatc) + inc_count, parent_mismatc)
    
    def updateBuffer(self, parent_buffer, new_buffer, parent_mismatch=0):
        parent_buffer[self.getOffset(parent_buffer, 
                                     parent_mismatch) + parent_mismatch:
                      self.getOffset(parent_buffer, 
                                     parent_mismatch) + parent_mismatch + self.getLength(parent_buffer, 
                                                                                         parent_mismatch)] = new_buffer
    
    def getOffsetRef(self, parent_buffer, parent_mismatch=0):
        if self.offset_position is not None:
            return DataPiece(parent_buffer, self.offset_position, self.offset_length, parent_mismatch)
        else:
            return None
    
    def getCountRef(self, parent_buffer, parent_mismatch=0):
        if self.count_position is not None:
            return DataPiece(parent_buffer, self.count_position, self.count_length, parent_mismatch)
        else:
            return None
    
    def getLengthRef(self, parent_buffer, parent_mismatch=0):
        if self.length_position is not None:
            return DataPiece(parent_buffer, self.length_position, self.length_length, parent_mismatch)
        else:
            return None

In [4]:
class BaseSegment(BaseOps):
    
    class Subs(SegmentStructure):
        pass
    
    class Record(BaseTable):
        pass
    
    def __init__(self, filename, name, parent, buffer, header_length, mismatch, 
                 offset_ref, count_ref, length_ref, record_length, spacing,
                 abs_count=None, abs_length=None, extra_segment=None):
        self.filename = filename
        self.name = name
        self.parent = parent
        self.buffer = buffer
        self.header_length = header_length  # anything before offset 0 starts
        self.mismatch = mismatch  #  extra mismatch on offset
        self.offset_ref = offset_ref
        self.count_ref = count_ref
        self.length_ref = length_ref
        self.record_length = record_length
        self.spacing = spacing   #  the first sub-segment offset from file beginning, adding length of this sub-segment resulted in offset for the next subsegment
        self.abs_count = abs_count
        self.abs_length = abs_length
        self.extra_segment = extra_segment  # space between records and sub-segments that keeps sub-segment offset and count reference
        self.header = None
        self.tail = None
        self.previous = None
        self.next = None
        self.children = list()
        self._read()
        self.split()

        
    @property
    def count(self):
        if self.abs_count:
            return self.abs_count
        elif self.count_ref is not None:
            return self.count_ref.getValue()
        else:
            return None
    
    @property
    def offset(self):
        if self.offset_ref:
            return self.offset_ref.getValue()
        else:
            return 0
        
    @property
    def readLength(self):
        if self.abs_length is not None:
            return self.abs_length
        elif self.length_ref is not None:
            return self.length_ref.getValue()
        elif self.count is not None:
            return self.count * self.record_length
        else:
            return None
        
        
    @property
    def calLength(self):
        _len = len(b''.join(self.records))
        if self.header is not None:
            _len += len(self.buffer[self.header])
        if self.tail is not None:
            _len += len(self.buffer[self.tail])
        if self.extra_segment is not None:
            _len += len(self.buffer[self.extra_segment])
        for _seg in self.children:
            _len += _seg.calLength
        return _len
    
    @property
    def combined_records(self):
        return b''.join(self.records)
     
        
    def _read(self):
        self.records = list()
        if self.count is not None:
            if self.count > 1000:
                print(self.parent.name, self.name, "Count:", self.count)
                pass
            for _i in range(self.count):
                _start = self.header_length + self.record_length * _i
                _end = _start + self.record_length
                _record = self.buffer[_start: _end]
                self.records.append(_record)
    
    @property
    def data(self):
        _cols = self.Record.__members__.keys()
        self._data = pd.DataFrame(columns=_cols)
        for _record in self.records:
            _dict = dict(zip(_cols, [_.getRepr(_record) for _ in self.Record]))
            self._data = self._data.append(_dict, ignore_index=True) 
        return self._data
        
    
    def split(self):
        self.children = list()
        _end = 0
        for _index, _item in enumerate(self.Subs):
            _name = _item.name
            _offset = _item.getOffset(self.buffer, self.header_length)
            _length = _item.getLength(self.buffer, self.header_length)
            setattr(self, _name, 
                    getattr(Appendix, _name)(self.filename, _name, self, 
                                             _item.getBufferCopy(self.buffer, self.header_length),
                                             _item.header_length, 
                                             _item.mismatch,
                                             _item.getOffsetRef(self.buffer, self.header_length),
                                             _item.getCountRef(self.buffer, self.header_length),
                                             _item.getLengthRef(self.buffer, self.header_length),
                                             _item.record_length,
                                             _item.spacing,
                                             _item.abs_count,
                                             _item.abs_length,
                                             _item.extra_segment
                                            )
                   )
            self.children.append(getattr(self, _name))    
            if _end < _offset + _length:
                _end = _offset + _length
        if _end == 0:
            _end = self.readLength
        self.header = slice(None, self.header_length)
        self.tail = slice(_end, None)
        
    '''    
    def _pack(self):
        _sub_buffer = bytearray()
        _extra_spacing = len(self.combined_records) + len(self.buffer[self.header])
        if self.extra_segment is not None:
            _extra_spacing += len(self.buffer[self.extra_segment])
        for _sub in self.children:
            if _sub.offset_ref is not None:
                if _sub.count == 0:
                    _sub.offset_ref.setValue(0)
                else:
                    _sub.offset_ref.setValue(len(_sub_buffer) + _extra_spacing)
            if _sub.count_ref is not None:
                _sub.count_ref.setValue(_sub.count)
            if _sub.length_ref is not None:
                _sub.length_ref.setValue(_sub.calLength())
            _sub_buffer.extend(_sub.pack())
        _buffer = bytearray()
        _buffer.extend(self.buffer[self.header])
        _buffer.extend(self.combined_records)
        if self.extra_segment is not None:
            _buffer.extend(self.buffer[self.extra_segment])
        _buffer.extend(_sub_buffer)
        _buffer.extend(self.buffer[self.tail])
        return _buffer
    '''

    def pack(self, *spacing):
        if not spacing:
            spacing = self.spacing
        else:
            spacing = spacing[0]
        _sub_buffer = bytearray()
        _spacing = len(self.combined_records) + len(self.buffer[self.header])
        if self.extra_segment is not None:
            _spacing += len(self.buffer[self.extra_segment])
        for _sub in self.children:
            _sub_buffer.extend(_sub.pack(len(_sub_buffer) + _spacing))
        if self.offset_ref is not None:
            if self.count == 0:
                pass
            else:
                self.offset_ref.setValue(spacing)
        if self.count_ref is not None:
            self.count_ref.setValue(self.count)
        if self.length_ref is not None:
            self.length_ref.setValue(self.calLength)
        _buffer = bytearray()
        _buffer.extend(self.buffer[self.header])
        _buffer.extend(self.combined_records)
        if self.extra_segment is not None:
            _buffer.extend(self.buffer[self.extra_segment])
        _buffer.extend(_sub_buffer)
        _buffer.extend(self.buffer[self.tail])
        return _buffer
    
    @staticmethod
    def searchName(_value, table):
        if not isinstance(_value, str):
            _value = str(_value, encoding='UTF-8')
        _value = _value.replace('\x00', '')
        for _cls in table.__dict__.values():
            if isinstance(_cls, EnumMeta):
                for _item in _cls:
                    if _item.value == _value:
                        return _item.name, _cls.__name__
        else:
            return _value, None

In [5]:
class Appendix:
    
    class KnownSpell(BaseSegment):
        
        class Record(BaseTable):

            SPELL = (0x0000, 8)
            LEVEL = (0x0008, 2)
            TYPE = (0x000a, 2)
            
        SPELL_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        SPELL_PATTERN = re.compile(r'(?P<type>\w{4})(?P<level>\d)(?P<id>\d+)')
        SPELL_TYPE = {'SPWI': 1,
                      'SPCL': 3,
                      'SPIN': 2,
                      'SPPR': 0,
                      'SPSD': 4,
                      1: 'Wizard',
                      0: 'Cleric',
                      2: 'Innate',
                      3: 'Class',
                      4: 'Special'}
            
        def prepareTemplate(self, spell):
            _string = re.match(self.SPELL_PATTERN, spell.value)
            _template = self.SPELL_TEMPLATE[:]
            _spell = self.bytes2Num(bytearray(spell.value, encoding='UTF-8'))
            _level = int(_string.group('level')) - 1
            if _level < 0:
                raise('Wait to figure out Lv.10 spells!')
            _type = self.SPELL_TYPE[_string.group('type')]
            self.Record.SPELL.setValue(_template, _spell)
            self.Record.LEVEL.setValue(_template, _level)
            self.Record.TYPE.setValue(_template, _type)
            return _template
        
        def _insert(self, spell):
            _count = 0
            _template = self.prepareTemplate(spell)
            if not _template in self.records:
                self.records.append(_template)
                _count += 1
            return _count
        
        def _delete(self, spell):
            _count = 0
            _template = self.prepareTemplate(spell)
            if _template in self.records:
                self.records.pop(self.records.index(_template))
                _count += 1
            return _count
        
        def insert(self, spell):
            _count = 0
            if isinstance(spell, EnumMeta):
                for _spell in spell:
                    _count += self._insert(_spell)
            else:
                _count = self._insert(spell)
            if self.count_ref:
                self.count_ref.incValue(_count)
            self.sort()
                
        def delete(self, spell):
            _count = 0
            if isinstance(spell, EnumMeta):
                for _spell in spell:
                    _count += self._delete(_spell)
            else:
                _count = self._delete(spell)
            if self.count_ref:
                self.count_ref.incValue(-_count)
            self.sort()
                
        def sort(self):
            self.records = sorted(self.records, key=lambda x: (self.Record.TYPE.getValue(x, 0),
                                                               self.Record.LEVEL.getValue(x, 0)))
            
        @property
        def data(self):
            self._data = super().data
            _name = self.Record.SPELL.name
            _length = self.Record.SPELL.length
            self._data[_name] = self._data[_name].apply(lambda x: self.searchName(self.num2Bytes(int(x), _length),
                                                                                  table=Spells)[0])
            return self._data
                    
    class MemSpell(BaseSegment):
        
        SPELL_TYPE = {'SPWI': 1,
                      'SPCL': 3,
                      'SPIN': 2,
                      'SPPR': 0,
                      'SPSD': 4,
                      1: 'Wizard',
                      0: 'Cleric',
                      2: 'Innate',
                      3: 'Class',
                      4: 'Special'}
        
        class Storage(Enum):
            CLERIC = 0, 7
            WIZARD = 7, 9
            INNATE = 16, 1
            SPECIAL = 17, 1
        
        class Record(BaseTable):
            LEVEL = (0x0000, 2)
            BASE_NUM = (0x0002, 2)
            EFF_NUM = (0x0004, 2)
            TYPE = (0x0006, 2)
            INDEX = (0x0008, 4)
            COUNT = (0x000c, 4)
            
        @property
        def data(self):
            _data = super().data
            _data['LEVEL'] = _data['LEVEL'].astype(int)
            _data['LEVEL'] = _data['LEVEL'] + 1
            return _data
        
        def setNumber(self, _type, _level, _number):
            if _level <= _type.value[1]:
                _index = _type.value[0] + _level - 1
                _base = self.Record.BASE_NUM.getValue(self.records[_index])
                _eff = self.Record.EFF_NUM.getValue(self.records[_index])
                self.Record.BASE_NUM.setValue(self.records[_index], _number)
                self.Record.EFF_NUM.setValue(self.records[_index], _number + _eff - _base)
                
        def optimize(self, _number=100):
            for _record in self.records:
                _base = self.Record.BASE_NUM.getValue( _record)
                _eff = self.Record.EFF_NUM.getValue( _record)
                self.Record.BASE_NUM.setValue(_record, _number)
                self.Record.EFF_NUM.setValue(_record, _number + _eff - _base)
                
        def updateMem(self, _summary):
            _len = len(self.records)
            for _index, _record in enumerate(_summary[:_len]):
                self.Record.INDEX.setValue(self.records[_index], _record[0])
                self.Record.COUNT.setValue(self.records[_index], _record[1])  
               
    class MemorizedSpell(BaseSegment):
        
        SPELL_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
        SPELL_PATTERN = re.compile(r'(?P<type>\w{4})(?P<level>\d)(?P<id>\d+)')
        SPELL_TYPE = {'SPWI': 1,
                      'SPCL': 3,
                      'SPIN': 2,
                      'SPPR': 0,
                      'SPSD': 4,
                      1: 'Wizard',
                      0: 'Cleric',
                      2: 'Innate',
                      3: 'Class',
                      4: 'Special'}
        STORAGE_VOLUME = {0: range(1,8),
                          1: range(1,10),
                          2: range(0,1),
                          3: range(1,2),
                          4: range(0,1)}
        
        class Record(BaseTable):
            SPELL = (0x0000, 8)
            MEM = (0x0008, 4)
            
        def sort(self):
            self.records = sorted(self.records, key=lambda x: self._parseRecord(x)[1])
            
        def prepareTemplate(self, spell, mem=1):
            _template = self.SPELL_TEMPLATE[:]
            _spell = self.bytes2Num(bytearray(spell.value, encoding='UTF-8'))
            self.Record.SPELL.setValue(_template, _spell)
            self.Record.MEM.setValue(_template, mem)
            return _template
        
        def _parseRecord(self, _record):
            _dict = dict()
            _name = self.Record.SPELL.getString(_record)
            _match = re.match(self.SPELL_PATTERN, _name)
            _dict['SPELL'], _dict['CATEGORY'] = self.searchName(_name, Spells)
            _dict['LEVEL'] = int(_match.group('level'))
            _dict['TYPE'] = int(self.SPELL_TYPE[_match.group('type')])
            _dict['MEM'] = int(self.Record.MEM.getRepr(_record))
            _dict['ID'] = _match.group('id')
            return _dict, [self.SPELL_TYPE[_dict['TYPE']], _dict['LEVEL'], _dict['ID']]
        
        @property
        def data(self):
            _cols = self.Record.__members__.keys()
            self._data = pd.DataFrame(columns=_cols)
            for _record in self.records:
                _dict = self._parseRecord(_record)[0]
                self._data = self._data.append(_dict, ignore_index=True) 
            if not self._data.empty:
                self._data['TYPE'] = self._data['TYPE'].astype(int)
                self._data['LEVEL'] = self._data['LEVEL'].astype(int)
                self._data['MEM'] = self._data['MEM'].astype(int)
            return self._data
        
        def _memorize(self, spell, _num=1):
            for i in range(_num):
                self.records.append(self.prepareTemplate(spell, 1))
      
        def memorize(self, spell, _num=1):
            self._memorize(spell, _num)       
            self.sort()
            self.updateMem()
            
        def memorize2(self, spells, forget=True):
            if forget:
                self.forgetAll()
            for _spell, _num in spells:
                self._memorize(_spell, _num)
            self.sort()
            self.updateMem()
            
        def forget(self, spell, _num=1):
            _template = self.prepareTemplate(spell, 1)
            for i in range(_num):
                if _template in self.records:
                    self.records.pop(self.records.index(_template))
            self.sort() 
            self.updateMem()
            
        def forgetAll(self, spells=['Cleric', 'Wizard']):
            if spells is None or spells == []:
                self.records = list()
            else:
                _pop = list()
                for _index, _record in enumerate(self.records):
                    _type = self.SPELL_TYPE[self._parseRecord(_record)[0]['TYPE']]
                    if _type in spells:
                        _pop.append(_index)
                for _index in _pop[::-1]:
                    self.records.pop(_index)
                self.sort()
            self.updateMem()
            
        def updateMem(self):
            _summary = list()
            if self.data.empty:
                for _level in self.STORAGE_VOLUME[0]:
                    _summary.append((0, 0))
                for _level in self.STORAGE_VOLUME[1]:
                    _summary.append((0, 0))
                _summary.append((0, 0))
            else:
                _dict = self.data.groupby(['TYPE', 'LEVEL']).count()['MEM'].to_dict()
                _offset = 0
                for _level in self.STORAGE_VOLUME[0]:
                    _count = int(_dict.get((0, _level), 0))
                    _summary.append((_offset, _count))
                    _offset += _count
                for _level in self.STORAGE_VOLUME[1]:
                    _count = int(_dict.get((1, _level), 0))
                    _summary.append((_offset, _count))
                    _offset += _count
                _count = 0
                for k, v in _dict.items():
                    if k[0] > 1:
                        _count += v
                _summary.append((_offset, int(_count)))
            self.parent.MemSpell.updateMem(_summary)
      
    
    class ItemSlot(BaseSegment):
        
        class Position(BaseTable):
        
            HELMET = (0x0000, 2)
            ARMOR = (0x0002, 2)
            SHIELD = (0x0004, 2)
            GLOVES = (0x0006, 2)
            RINGS = (0x0008, 4)
            AMULET = (0x000c, 2)
            BELT = (0x000e, 2)
            BOOTS = (0x0010, 2)
            WEAPONS = (0x0012, 2)
            EXTRAWEAPONS = (0x0016, 6)
            QUIVERS = (0x001a, 6)
            EXTRAQUIVER = (0x0020, 2)
            CLOAK = (0x0022, 2)
            QUICK = (0x0024, 6)
            INVENTORY = (0x002a, 32) 
            MAGICWEAPON = (0x004a, 2)
            SELECTEDWEAPON = (0x004c, 2)
            SELECTEDABILITY = (0x004e, 2)
            
        class Category(BaseOps):
            
            def __init__(self, capacity):
                self.index = list()
                self.item = list()
                self.code = list()
                self.capacity = capacity
                
            def add(self, _index, _item):
                if len(self.index) < self.capacity:
                    _code = self.num2Bytes(_index, 2)
                    self.code.append(_code)
                    self.item.append(_item)
                    self.index.append(_index)
                    return True
                else:
                    return False
                    
            def fillna(self):
                self.code.extend([b'\xff\xff'] * (self.capacity - len(self.index)))
                return self.code
            
        def delete(self, index):
            self.records[index] = b'\xff\xff'
            
        def place(self, inventory, items):
            self.records = list()
            
            self.helmet = self.Category(1)
            self.armor = self.Category(1)
            self.shield = self.Category(1)
            self.gloves = self.Category(1)
            self.rings = self.Category(2)
            self.amulet = self.Category(1)
            self.belt = self.Category(1)
            self.boots = self.Category(1)
            self.weapons = self.Category(1)
            self.extraweapons = self.Category(3)
            self.quivers = self.Category(3)
            self.extraquiver = self.Category(1)
            self.cloak = self.Category(1)
            self.quick = self.Category(3)
            self.inventory = self.Category(16)
            self.magicweapon = self.Category(1)
            self.selectedweapon = self.Category(1)
            self.selectedability = self.Category(1)

            occupied = {'LONGBOWS': self.weapons,
                        'ONE_HANDED_SWORDS': self.weapons,
                        'STAVES': self.weapons,
                        'FAMILIARS': self.inventory,
                        'BASTARDSWORDS': self.weapons,
                        'WARDSTONE': self.inventory,
                        'SHORTBOWS': self.weapons,
                        'LONGSWORDS': self.weapons,
                        'BRACERS': self.gloves,
                        'CLUBS': self.weapons,
                        'Exceptions': self.inventory,
                        'INSTRUMENTS': self.inventory,
                        'AXES': self.weapons,
                        'THROWINGAXES': self.weapons,
                        'FLAILS': self.weapons,
                        'MEDIUMSHIELDS': self.shield,
                        'ELVENCHAINS': self.armor,
                        'WANDS': self.weapons,
                        'KEYS': self.inventory,
                        'GEM': self.inventory,
                        'HEAVYCROSSBOWS': self.weapons,
                        'HALBERDS': self.weapons,
                        'RING2': self.rings,
                        'LARGESHIELDS': self.shield,
                        'COMPOSITEBOWS': self.weapons,
                        'SPEARS': self.weapons,
                        'BOLTS': self.quivers,
                        'PLOT_ITEMS': self.inventory,
                        'MACES': self.weapons,
                        'BAGS': self.inventory,
                        'FULLPLATES': self.armor,
                        'PLATEMAILS': self.armor,
                        'SPLINTMAILS': self.armor,
                        'STUDDEDLEATHERS': self.armor,
                        'MISC': self.inventory,
                        'BUCKLERS': self.shield,
                        'SMALLSHIELDS': self.shield,
                        'IWD': self.inventory,
                        'COMPONENTS': self.inventory,
                        'WAKIZASHI': self.weapons,
                        'BLUNT_WEAPONS': self.weapons,
                        'RODS': self.inventory,
                        'DAGGERS': self.weapons,
                        'CLOAKS': self.cloak,
                        'AMULETS': self.amulet,
                        'LEATHERS': self.armor,
                        'ARROWS': self.quivers,
                        'SHORTSWORDS': self.weapons,
                        'BELTS': self.belt,
                        'TWOHANDSWORDS': self.weapons,
                        'RINGS': self.rings,
                        'HAMMERS': self.weapons,
                        'SHIELDS': self.shield,
                        'DRAGON_SCALES': self.armor,
                        'SCIMITAR': self.weapons,
                        'DROW_ITEMS': self.inventory,
                        'SECRET_PANTALOON_STUFF':self.inventory,
                        'LIGHTCROSSBOWS': self.weapons,
                        'HELMS': self.helmet,
                        'POTIONS': self.inventory,
                        'BULLETS': self.quivers,
                        'NOTES': self.inventory,
                        'SLINGS': self.weapons,
                        'CHAINMAILS': self.armor,
                        'DARTS': self.quivers,
                        'QUIVERS': self.quivers,
                        'AMULETS2': self.amulet,
                        'BOOTS': self.boots,
                        'SCROLLS': self.inventory,
                        'BOOKS': self.inventory,
                        'ROBES': self.armor,
                        'TOMES': self.inventory,
                        }
            
            for _index, (_category, _item) in enumerate(zip(inventory, items)):
                if _category in occupied.keys():
                    if not occupied[_category].add(_index, _item):
                        for _other in [self.inventory, self.quick, self.quivers, self.shield, self.helmet, 
                                       self.armor, self.gloves, self.rings, self.amulet, self.belt, self.boots,
                                       self.weapons, self.cloak, self.extraweapons]:
                            if _other.add(_index, _item):
                                break
                else:
                    self.inventory.add(_index, _item)
            '''
            extra_items = list()
            for _index, (_category, _item) in enumerate(zip(inventory, items)):
                if _category in occupied.keys():
                    if not occupied[_category].add(_index, _item):
                        extra_items.append((_index, _category, _item))
                else:
                    extra_items.append((_index, _category, _item))
            for _index, _category, _item in extra_items:
                for _other in [self.inventory, self.quick, self.quivers, self.shield, self.helmet, 
                               self.armor, self.gloves, self.rings, self.amulet, self.belt, self.boots,
                               self.weapons, self.cloak, self.extraweapons]:
                    if _other.add(_index, _item):
                        break
            '''        
            for _item in self.Position:
                _category = getattr(self, _item.name.lower())
                self.records.extend(_category.fillna())
            
    class Item(BaseSegment):
        
        GOODITEMS =  [Items.AMULETS.Amulet_of_Power, 
                      Items.AMULETS.Amulet_of_the_Master_Harper,
                      Items.AXES.Axe_of_the_Unyielding_PLUS5,
                      Items.BELTS.Belt_of_Inertial_Barrier,
                      Items.FLAILS.Flail_of_Ages_PLUS5,
                      Items.BOOTS.Boots_of_Speed,
                      Items.SHORTBOWS.Short_Bow_of_Gesen,
                      Items.LONGBOWS.Taralash_PLUS5,
                      Items.BRACERS.Gauntlets_of_Extraordinary_Specialization,
                      Items.CHAINMAILS.Aslyferund_Elven_Chain_PLUS5,
                      Items.CLOAKS.Cloak_of_Mirroring,
                      Items.CLOAKS.Cloak_of_Bravery,
                      Items.ROBES.Robe_of_the_Good_Archmagi,
                      Items.ROBES.Robe_of_the_Neutral_Archmagi,
                      Items.LIGHTCROSSBOWS.Firetooth_PLUS5,
                      Items.DAGGERS.Dagger_of_the_Star_PLUS5,
                      Items.HALBERDS.Ravager_PLUS6,
                      Items.HAMMERS.Runehammer_PLUS5,
                      Items.HELMS.Circlet_of_Netheril_1,
                      Items.HELMS.Kiels_Helmet,
                      Items.HELMS.Helm_of_Charm_Protection,
                      Items.FULLPLATES.Enkidus_Full_Plate_PLUS3,
                      Items.RINGS.Ring_of_Wizardry,
                      Items.RINGS.Ring_of_Holiness,
                      Items.RINGS.Ring_of_Gaxx,
                      Items.RINGS.Ring_of_Acuity,
                      Items.RINGS.Heartwood_Ring,
                      Items.RINGS.Ring_of_AntiVenom,
                      Items.SMALLSHIELDS.Reflection_Shield_PLUS1,
                      Items.SMALLSHIELDS.Shield_of_Harmony_PLUS2,
                      Items.SLINGS.Erinne_Sling_PLUS5,
                      Items.SPEARS.Spear_of_the_Unicorn_PLUS2,
                      Items.STAVES.Staff_of_the_Magi,
                      Items.STAVES.Staff_of_the_Ram_PLUS6,
                      Items.LONGSWORDS.Blackrazor_PLUS3,
                      Items.SCIMITAR.Belm_PLUS2,
                      Items.LONGSWORDS.Adjatha_the_Drinker_PLUS2,
                      Items.WAKIZASHI.Celestial_Fury_PLUS3,
                      Items.SHORTSWORDS.Short_Sword_of_Mask_PLUS5,
                      Items.LONGSWORDS.Angurvadal_PLUS5,
                      Items.LONGSWORDS.The_Answerer_PLUS4,
                      Items.SCIMITAR.Scimitar_PLUS5_Defender,
                      Items.TWOHANDSWORDS.Psions_Blade_PLUS5,
                      Items.QUIVERS.Bag_of_Plenty_PLUS2,
                      Items.QUIVERS.Case_of_Plenty_PLUS2,
                      Items.QUIVERS.Quiver_of_Plenty_PLUS2]

        GOODITEMS = [Items.RINGS.Ring_of_Wizardry,
                     Items.RINGS.Ring_of_Acuity,
                     Items.RINGS.Ring_of_Holiness,
                     Items.BRACERS.Gauntlets_of_Weapon_Expertise,
                     Items.BOOTS.Boots_of_Speed,
                     Items.AMULETS.Amulet_of_Power,
                     Items.FULLPLATES.Plate_of_the_Dark_PLUS1,
                     Items.SHORTBOWS.Eagle_Bow,
                     Items.ROBES.Robe_of_the_Good_Archmagi,
                     Items.HEAVYCROSSBOWS.Heavy_Crossbow_of_Accuracy,
                     Items.LIGHTCROSSBOWS.Light_Crossbow_of_Speed,
                     Items.LONGBOWS.Long_Bow_of_Marksmanship,
                     Items.SCIMITAR.Scimitar_PLUS5_Defender,
                     Items.CHAINMAILS.Elven_Chain_Mail,
                     Items.HAMMERS.War_hammer_PLUS2_Ashideena,
                     Items.AXES.Klogarath_PLUS4,
                     Items.QUIVERS.Bag_of_Plenty_PLUS1,
                     Items.QUIVERS.Case_of_Plenty_PLUS1,
                     Items.QUIVERS.Quiver_of_Plenty_PLUS1,
                     Items.BELTS.Belt_of_Inertial_Barrier,
                     Items.CLOAKS.Cloak_of_Bravery,
                     Items.CLOAKS.Cloak_of_NonDetection,
                     Items.STAVES.Narbucchad_Demise_PLUS5,
                     Items.TWOHANDSWORDS.Two_Handed_Sword_PLUS3,
                     Items.SLINGS.Sling_Dragonbane_PLUS3,
                     Items.BUCKLERS.Shield_of_Devotion,
                     Items.BUCKLERS.Tarloc_Contingency_PLUS1,
                     Items.LARGESHIELDS.Pellans_Shield_PLUS2,
                     Items.SCIMITAR.Scimitar_PLUS3_Frostbrand,
                     Items.LARGESHIELDS.Sartessa_Vengeance,
                     Items.HELMS.Circlet_of_the_Archmage,
                     Items.HELMS.Helm_of_Charm_Protection,
                     Items.HELMS.Pale_Green_Ioun_Stone,
                     Items.CLOAKS.Cloak_of_Balduran
                    ]
        
        AVOIDITEMS = [Items.LARGESHIELDS.Sartessa_Vengeance]
    
        class Record(BaseTable):
            NAME = (0x0000, 8)
            EXPIRATION = (0x0008, 1)
            ELAPSED = (0x0009, 1)
            QUANTITY_1 = (0x000a, 2)
            QUANTITY_2 = (0x000c, 2)
            QUANTITY_3 = (0x000e, 2)
            IDENTIFIED = (0x000f, 1)
            UNSTEALABLE = (0x0010, 1)
            STOLEN = (0x0011, 1)
            UNDROPPABLE = (0x0012, 1)

        MAX_COUNT = 37
        MIN_COUNT = 33

        ITEM_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])
        
        LAUNCHER_CLASS = [Items.COMPOSITEBOWS, Items.DARTS, Items.HEAVYCROSSBOWS, 
                          Items.LIGHTCROSSBOWS, Items.LONGBOWS, Items.SHORTBOWS, Items.SLINGS]
        LAUNCHERS = [_ for cls in LAUNCHER_CLASS for _ in cls.__members__.values()]                                                    
         

        def removeLauncher(self, items):
            return [_ for _ in items if not _ in self.LAUNCHERS]
        
        @property
        def data(self):
            _col = list(self.Record.__members__.keys()) + ['CATEGORY']
            self._data = pd.DataFrame(columns=_col)
            for _record in self.records:
                _dict = dict()
                for _item in self.Record:
                    if _item == self.Record.NAME:
                        _dict[self.Record.NAME.name], _dict['CATEGORY'] = self.searchName(_item.getBytes(_record),
                                                                                          table=Items)
                    else:
                        _dict[_item.name] = _item.getValue(_record)
                self._data = self._data.append(_dict, ignore_index=True)
            for col in self._data.columns:
                if not col in [self.Record.NAME.name, 'CATEGORY']:
                    self._data[col] = self._data[col].astype(int)
            return self._data
        
        def _insert(self, item, quantity=1, unique=False):
            _template = self.ITEM_TEMPLATE[:]
            self.Record.NAME.setValue(_template, self.bytes2Num(bytearray(item.value, encoding='UTF-8')))
            self.Record.QUANTITY_1.setValue(_template, quantity) 
            if self.count < self.MAX_COUNT:
                if unique == False or not _template in self.records:
                    self.records.insert(0, _template)
                    if self.count_ref is not None:
                        self.count_ref.incValue(1)
                    return True
                else:
                    return False
            else:
                if unique == False or not _template in self.records:
                    self.records = self.records[:-1]
                    self.records.insert(0, _template)
                    return True
                else:
                    return False
                    
        def insert(self, item, quantity=1, unique=False):
            if self._insert(item, quantity, unique) == True:
                self.place()
        
        def delete(self, item, unique=False):
            _code = self.bytes2Num(item.value.encode(), self.Record.NAME.length)
            _items = [self.Record.NAME.getValue(_record) for _record in self.records]
            _index = [_i for _i, _item in enumerate(_items) if _item == _code]
            if _index:
                if unique == True:
                    _index = _index[0:1]
                for _i in _index:
                    self.records.pop(_i)
                if self.count_ref is not None:
                    self.count_ref.incValue(-len(_index))  
                    
        def _deleteAll(self):
            self.records = list()
            if self.count_ref is not None:
                self.count_ref.setValue(0)
                    
        def place(self):
            categories = self.data['CATEGORY']
            items = self.data['NAME']
            self.parent.ItemSlot.place(categories, items)
            
        def optimize(self, size=None, avoid=False):
            if avoid:
                _pool = [_ for _ in self.GOODITEMS if not _ in self.AVOIDITEMS]
            else:
                _pool = self.GOODITEMS
            _len = len(_pool)
            _pool = _pool * (self.MIN_COUNT // _len) + sample(_pool, self.MIN_COUNT % _len)
            _leftover = list(set(self.GOODITEMS).difference(_pool))
            _leftover = self.removeLauncher(sample(_leftover, len(_leftover)) + sample(self.GOODITEMS, _len))
            _pool = _pool + _leftover[:self.MAX_COUNT - self.MIN_COUNT]
            if size is None or size > self.MAX_COUNT:
                size = self.MAX_COUNT
            for _item in _pool[:size][::-1]:
                self._insert(_item) 
            self.place()

    class Effect(BaseSegment):
        
        class WeaponOptima(Enum):
            BastardSword = 5
            LongSword = 5
            ShortSword = 5
            Axe = 5
            TwoHandedSword = 5
            Katana = 5
            ScimitarWakizashiNinjaTo = 5
            Dagger = 5
            WarHammer = 5
            Spear = 5
            Halberd = 5
            FlailMorningstar = 5
            Mace = 5
            QuarterStaff = 5
            Crossbow = 5
            LongBow = 5
            ShortBow = 5
            Darts = 5
            Sling = 5
            #Blackjack = 5
            #Gun = 5
            #MartialArts = 5
            TwoHandedWeaponSkill = 2
            SwordandShieldSkill = 2
            SingleWeaponSkill = 2
            TwoWeaponSkill = 3
            Club = 5
            #ExtraProficiency2 = 5
            #ExtraProficiency3 = 5
            #ExtraProficiency4 = 5
            #ExtraProficiency5 = 5
            #ExtraProficiency6 = 5
            #ExtraProficiency7 = 5
            #ExtraProficiency8 = 5
            #ExtraProficiency9 = 5
            #ExtraProficiency10 = 5
            #ExtraProficiency11 = 5
            #ExtraProficiency12 = 5
            #ExtraProficiency13 = 5
            #ExtraProficiency14 = 5
            #ExtraProficiency15 = 5
            #ExtraProficiency16 = 5
            #ExtraProficiency17 = 5
            #ExtraProficiency18 = 5
            #ExtraProficiency19 = 5
            #ExtraProficiency20 = 5
    
        WEAPON_OPCODE = Tables.Effects.Stat_Proficiency_Modifier_Variants_IWD1.value
        WEAPON_STRING = Tables.Effects.Stat_Proficiency_Modifier_Variants_IWD1.name
        WEAPON_TEMPLATE = bytearray([0, 0, 0, 0, 0, 0, 0, 0, 233, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 95, 
                                     0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 255, 
                                     255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 255, 255, 255, 255, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                                     0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

        class Record(BaseTable):
            OPCODE = (0x0008, 4, False, 'little', 'Effects')
            TARGET = (0x000c, 4, False, 'little', 'Target')
            POWER = (0x0010, 4)
            PARAM_1 = (0x0014, 4, False, 'little', None, 5)
            PARAM_2 = (0x0018, 4, False, 'little', 'WeaponType')
            TIME = (0x001c, 4, False, 'little', 'Time')

        
        def insert(self, weapon, *proficiency):
            if not proficiency:
                proficiency = getattr(self.WeaponOptima, weapon.name).value
            else:
                proficiency = proficiency[0]
            _weapons = [self.Record.PARAM_2.getValue(_record) for _record in self.records if self.Record.OPCODE.getValue(_record) == self.WEAPON_OPCODE]
            _index = [_i for _i, _weapon in enumerate(_weapons) if _weapon == weapon.value]
            if _index:
                self.Record.PARAM_1.setValue(self.records[_index[0]], proficiency)
            else:
                _template = self.WEAPON_TEMPLATE[:]
                self.Record.PARAM_1.setValue(_template, proficiency)
                self.Record.PARAM_2.setValue(_template, weapon.value)
                self.records.insert(0, _template)
                if self.count_ref is not None:
                    self.count_ref.incValue(1)
                    
        def optimize(self, more_weapon=True):
            if more_weapon:
                for _weapon in Tables.WeaponType:
                    self.insert(_weapon)
            else:
                _weapons = [_record for _record in self.records if self.Record.OPCODE.getValue(_record) == self.WEAPON_OPCODE]
                for _weapon in _weapons:
                    _code = self.Record.PARAM_2.getValue(_weapon)
                    _name = Tables.WeaponType(_code).name
                    _proficiency = getattr(self.WeaponOptima, _name).value
                    self.Record.PAMAM_1.setValue(_weapon, _proficiency) 
                    
        @property
        def weapons(self):
            return super().data[super().data['OPCODE'] == self.WEAPON_STRING]
        
    class Globals(BaseSegment):
        
        class Record(BaseTable):
            NAME = (0x0000, 32)
            TYPE = (0x0020, 2)
            REF = (0x0022, 2)
            DWORD = (0x0024, 4)
            INT = (0x0028, 4, True)
            DOUBLE = (0x002c, 8)
            SCRIPT = (0x0034, 32)
        
        @property
        def data(self):
            _cols = self.Record.__members__.keys()
            self._data = pd.DataFrame(columns=_cols)
            for _record in self.records:
                _dict = dict(zip(_cols, [_.getRepr(_record) for _ in self.Record]))
                _dict['NAME'] = self.Record.NAME.getString(_record)
                self._data = self._data.append(_dict, ignore_index=True) 
            return self._data
        
        def changeGlobal(self, _name, _value):
            for _record in self.records:
                if _name == self.Record.NAME.getString(_record):
                    self.Record.INT.setValue(_record, _value)      

In [6]:
class CRE(BaseSegment):
    
    Record = Segments
    
    class Subs(SegmentStructure, Enum):
        KnownSpell = (0, 0, 0x02a0, 4, 0x02a4, 4, None, None, 12, 52)
        MemSpell = (0, 0, 0x02a8, 4, 0x02ac, 4, None, None, 16, 44)
        MemorizedSpell = (0, 0, 0x02b0, 4, 0x02b4, 4, None, None, 12, 36)
        ItemSlot = (0, 0, 0x02b8, 4, None, None, None, None, 2, 28, 40, 80)
        Item = (0, 0, 0x02bc, 4, 0x02c0, 4, None, None, 20, 24)
        Effect = (0, 0, 0x02c4, 4, 0x02c8, 4,None, None,  264, 16)
            
    @property
    def data(self):
        self._data = super().data.T
        self._data.columns = [self.name]
        return self._data[~self._data.index.str.contains('Ref|Count|Offset')]
    
    @property
    def weapons(self):
        return self.Effect.weapons
    
    @property
    def items(self):
        return self.Item.data
    
    @property
    def spells(self):
        return self.KnownSpell.data
    
    @property
    def mem(self):
        return self.MemSpell.data
    
    @property
    def memorized(self):
        return self.MemorizedSpell.data
    
    @property
    def effects(self):
        return self.Effect.data
    
    def optimize(self):
        for _item in Segments:
            _item.optimize(self.records[0]) 
            
    def optimize2(self):
        for _item in Segments:
            _item.optimize2(self.records[0])
            
    def setValue(self, item, new_value):
        if not isinstance(new_value, int):
            new_value = new_value.value       
        getattr(Segments, item.name).setValue(self.records[0], new_value) 
        
    def optimizeWeapon(self):
        self.Effect.optimize()
        
    def addSpell(self, spell):
        self.KnownSpell.insert(spell)
        
    def removeSpell(self, spell):
        self.KnownSpell.delete(spell)
        
    def addItem(self, item):
        self.Item.insert(item)
        
    def removeItem(self, item):
        self.Item.delete(item)
        
    def optimizeItem(self, size=None, avoid=False):
        self.Item.optimize(size=size, avoid=avoid)
        
    def optimizeMem(self, num=100):
        self.MemSpell.optimize(num)
        
    def memorize(self, spell, _num=1):
        self.MemorizedSpell.memorize(spell, _num)
        
    def memorize2(self, spells, forget=True):
        self.MemorizedSpell.memorize2(spells, forget)
        
    def forget(self, spell, _num=1):
        self.MemorizedSpell.forget(spell, _num)
        
    def forgetAll(self):
        self.MemorizedSpell.forgetAll()

In [7]:
class GAM(BaseOps):
    
    class Record(BaseTable): 
        GOLD = (0x0018, 4, False, 'little', None, 1000000)
        REPUTATION = (0x0054, 4, False, 'little', None, 200)
        CAP = (0x0064, 4, False, 'little', None, 100)
        
    class COUNT(BaseTable):   
        PARTY_ = (0x001c, 2)
        PARTY = (0x0024, 4)
        NPC = (0x0034, 4)
    
    class ZIPOFFSET(BaseTable):
        COMPRESSED = (0x0012, 4)
        UNCOMPRESSED = (0x0016, 4)
        
    class OFFSET(BaseTable):  
        PARTY = (0x0020, 4)
        INVENTORY = (0x0028, 4)
        NPC = (0x0030, 4)
        GLOBALS = (0x0038, 4)
        FAMILIAR_EXTRA = (0x0048, 4)
        JOURNAL = (0x0050, 4)
        FAMILIAR_INFO = (0x0068, 4)
        STORED_LOCATION = (0x006c, 4)
        POCKET_PLANE = (0x0078, 4)
    
    class STRUCT(SegmentStructure):
        PARTY = (0, 0, 0x0020, 4, 0x0024, 4, None, None, 352, 0)
        INVENTORY = (0, 0, 0x0028, 4, 0x002c, 4, None, None, None, 0)
        NPC = (0, 0, 0x0030, 4, 0x0034, 4, None, None, 352, 0)
        GLOBALS = (0, 0, 0x0038, 4, 0x003c, 4, None, None, 84, 0)
        FAMILIAR_EXTRA = (0, 0, 0x0048, 4, None, None, None, None, None, 0)
        JOURNAL = (0, 0, 0x0050, 4, 0x004c, 4, None, None, 12, 0)
        FAMILIAR_INFO = (0, 0, 0x0068, 4, None, None, None, None, None, 0, 1, 592)
        STORED_LOCATION = (0, 0, 0x006c, 4, 0x0070, 4, None, None, 12, 0)
        POCKET_PLANE = (0, 0, 0x0078, 4, 0x007c, 4, None, None, 12, 0)
        
    class Subs(SegmentStructure):
        Globals = (0, 0, 0x0038, 4, 0x003c, 4, None, None, 84, 0)
        
    class CHAR(BaseTable):
        OFFSET = (0x0004, 4)
        LENGTH = (0x0008, 4)
        NAME_ = (0x000c, 8)
        NAME = (0x00c0, 32)

    NPC_ITEM_COUNT = 33
        
    def __init__(self, filename, buffer, header_slice):
        self.filename = filename
        self.buffer = buffer
        self.buffercopy = self.buffer[:]
        self.zipheader_slice = header_slice
        self.zipheader_length = len(self.buffer[self.zipheader_slice])
        self.file_slice = slice(self.zipheader_slice.stop, None)
        self.record_length = 180
        self.data_width = 4
        self.cre_record_length = 672
        self.cre_abs_count = 1
        self.cre_abs_length = None
        self.cre_extra_segment = slice(0x02a0, 0x02d4)
        self.children = list()
        self.PARTY = self.readChar('party')
        self.NPC = self.readChar('npc')
        self._head_slice = slice(None, self.PARTY[0].offset + self.zipheader_length)
        self._mid_slice = slice(self.PARTY[-1].offset + self.PARTY[-1].readLength + self.zipheader_length,
                                self.NPC[0].offset + self.zipheader_length)
        self._tail_slice = slice(self.NPC[-1].offset + self.NPC[-1].readLength + self.zipheader_length, None)
        self.split()
        
    def readChar(self, charset):
        _char = list()
        _offset = getattr(self.OFFSET, charset.upper()).getValue(self.buffer, self.zipheader_length)
        _count = getattr(self.COUNT, charset.upper()).getValue(self.buffer, self.zipheader_length)
        _record_length = getattr(self.STRUCT, charset.upper()).record_length
        for _index in range(_count):
            _start = _index * _record_length +  self.zipheader_length + _offset
            _end = _start + _record_length
            _record = self.buffer[_start: _end]
            _name = self.CHAR.NAME.getString(_record)
            _name_ = self.CHAR.NAME_.getString(_record)
            _name = _name or _name_
            _cre_offset_ref = DataPiece(self.buffer, 
                                        self.CHAR.OFFSET.position + _start, 
                                        self.CHAR.OFFSET.length, 
                                        0)
            _cre_length_ref = DataPiece(self.buffer, 
                                        self.CHAR.LENGTH.position + _start, 
                                        self.CHAR.LENGTH.length, 
                                        0)
            _cre_start = _cre_offset_ref.getValue() + self.zipheader_length
            _cre_length = _cre_length_ref.getValue()
            _buffer = self.buffer[_cre_start: _cre_start + _cre_length]
            _char.append(CRE(self.filename, _name, None, _buffer, 0, 0, _cre_offset_ref, None,
                             _cre_length_ref, self.cre_record_length, _cre_start, self.cre_abs_count,
                             self.cre_abs_length, self.cre_extra_segment))
        return _char
    
    def split(self):
        for _index, _item in enumerate(self.Subs):
            _name = _item.name
            _offset = _item.getOffset(self.buffer, self.zipheader_length)
            _length = _item.getLength(self.buffer, self.zipheader_length)
            setattr(self, _name, 
                    getattr(Appendix, _name)(self.filename, _name, self, 
                                             _item.getBufferCopy(self.buffer, self.zipheader_length),
                                             _item.header_length, 
                                             _item.mismatch,
                                             _item.getOffsetRef(self.buffer, self.zipheader_length),
                                             _item.getCountRef(self.buffer, self.zipheader_length),
                                             _item.getLengthRef(self.buffer, self.zipheader_length),
                                             _item.record_length,
                                             _item.spacing,
                                             _item.abs_count,
                                             _item.abs_length,
                                             _item.extra_segment
                                            )
                   )
            self.children.append(getattr(self, _name))         

        
    def _packPARTY(self, dragging=0, export=False):
        self.party_buffer = bytearray()
        _spacing = self._head_slice.stop - self.zipheader_length
        for _c in self.PARTY:
            _old_len = _c.readLength
            _new_len = _c.calLength
            if export == True:
                _p = _c.pack(_spacing + dragging)
                with open(_c.name + '_origin', 'wb') as _1, open(_c.name + '_packed', 'wb') as _2:
                    _1.write(_c.buffer)
                    _2.write(_p)
                self.party_buffer.extend(_p)
            else:
                self.party_buffer.extend(_c.pack(_spacing + len(self.party_buffer)))
            dragging += (_new_len - _old_len)
            _c.length_ref.setValue(_new_len)
        return dragging
    
    
    def _packNPC(self, dragging):
        self.npc_buffer = bytearray()
        _spacing = self._mid_slice.stop - self.zipheader_length + dragging
        for _c in self.NPC:
            _old_len = _c.readLength
            _new_len = _c.calLength
            self.npc_buffer.extend(_c.pack(_spacing + len(self.npc_buffer)))
            dragging += (_new_len - _old_len)
            _c.length_ref.setValue(_new_len)
        return dragging
    
    def _updateSubs(self):
        for _sub in self.Subs:
            _start = _sub.getOffset(self.buffer, self.zipheader_length)
            _end = _start + _sub.getLength(self.buffer, self.zipheader_length)
            _sub_buffer = getattr(self, _sub.name).combined_records
            self.buffer[_start + self.zipheader_length:
                        _end + self.zipheader_length] = _sub_buffer
             
    
    def pack(self):
        _dragging1 = self._packPARTY(0)
        _dragging2 = self._packNPC(_dragging1)
        self._updateSubs()
        self._incValue(self.OFFSET.INVENTORY, _dragging1)
        self._incValue(self.OFFSET.NPC, _dragging1)
        self._incValue(self.OFFSET.GLOBALS, _dragging2)
        self._incValue(self.OFFSET.FAMILIAR_EXTRA, _dragging2)
        self._incValue(self.OFFSET.JOURNAL, _dragging2)
        self._incValue(self.OFFSET.FAMILIAR_INFO, _dragging2)
        self._incValue(self.OFFSET.STORED_LOCATION, _dragging2)
        self._incValue(self.OFFSET.POCKET_PLANE, _dragging2)
        self.ZIPOFFSET.COMPRESSED.incValue(self.buffer, _dragging2)
        self.ZIPOFFSET.UNCOMPRESSED.incValue(self.buffer, _dragging2)
        self._buffer = self.buffer[self._head_slice]
        self._buffer.extend(self.party_buffer)
        self._buffer.extend(self.buffer[self._mid_slice])
        self._buffer.extend(self.npc_buffer)
        self._buffer.extend(self.buffer[self._tail_slice])
        self.buffer = self._buffer
        return self._buffer
    
    def _incValue(self, item, _incValue):
        _value = item.getValue(self.buffer, self.zipheader_length)
        if _value == 0:
            return
        try:
            item.incValue(self.buffer, _incValue, self.zipheader_length)
        except OverflowError:
            pass
        
    def optimize(self, NPC=False):
        for _item in self.Record:
            _item.optimize(self.buffer, self.zipheader_length)
        for _c in self.PARTY:
            _c.optimize()
        if NPC:
            for _c in self.NPC:
                _c.optimize2()
           
    def optimizeItem(self, NPC=False):
        for _c in self.PARTY:
            _c.optimizeItem()
        if NPC:
            for _c in self.NPC:
                _c.optimizeItem(size=self.NPC_ITEM_COUNT, avoid=True)
            
    def optimizeWeapon(self, NPC=False):
        for _c in self.PARTY:
            _c.optimizeWeapon() 
        if NPC:
            for _c in self.NPC:
                _c.optimizeWeapon()
            
    def optimizeMem(self, NPC=False, num_party=100, num_npc=10):
        for _c in self.PARTY:
            _c.optimizeMem(num_party) 
        if NPC:
            for _c in self.NPC:
                _c.optimizeMem(num_npc)            
            
    @property
    def globals(self):
        return self.Globals.data
    
    def changeGlobal(self, _name, _value):
        self.Globals.changeGlobal(_name, _value)
        
    def getNPC(self, _npc):
        if isinstance(_npc, int) and _npc<len(self.NPC):
            return self.NPC[_npc]
        elif isinstance(_npc, str):
            for _char in self.NPC:
                if _char.name and _char.name[1:].lower() == _npc[1:].lower():
                    return _char

In [8]:
class CHR:
    
    class Blocks(BaseTable):
        NAME = (0x0008, 32)
        CRE_OFFSET = (0x0028, 4)
        CRE_LENGTH = (0x002c, 4)
         
    def __init__(self, filename):
        self.filename = filename
        self.pwd = os.path.dirname(self.filename)
        self.file = os.path.basename(self.filename)
        self.zipfile = zipfile.ZipFile(self.filename)
        self.file_list = self.zipfile.namelist()
        self.buffer = bytearray(self.zipfile.read(self.file_list[1]))
        self.header_length = 0
        self.mismatch = 0
        self.record_length = 672
        self.spacing = 0
        self.abs_count = 1
        self.abs_length = None
        self.extra_segment = slice(0x02a0, 0x02d4)
        self.read()
        
    @property
    def name(self):
        _name = self.buffer[self.Blocks.NAME.position:
                            self.Blocks.NAME.position + self.Blocks.NAME.length]
        while _name[-1] == 0:
            _name = _name[:-1]
        return _name.decode('UTF-8')
    
    def read(self):
        _offset_ref = DataPiece(self.buffer, self.Blocks.CRE_OFFSET.position, self.Blocks.CRE_OFFSET.length)
        _length_ref = DataPiece(self.buffer, self.Blocks.CRE_LENGTH.position, self.Blocks.CRE_LENGTH.length)
        _start = _offset_ref.getValue()
        _length = _length_ref.getValue()
        _end = _start + _length
        self.header = slice(None, _start)
        self.tail = slice(_end, None)
        self.cre = CRE(self.filename, self.name, None, self.buffer[_start:_end], 
                       self.header_length, self.mismatch, _offset_ref, 
                       None, _length_ref, self.record_length, _start, self.abs_count,
                       self.abs_length, self.extra_segment)
    
    @property
    def weapons(self):
        return self.cre.weapons
    
    @property
    def items(self):
        return self.cre.items
    
    @property
    def spells(self):
        return self.cre.spells
    
    @property
    def data(self):
        return self.cre.data
    
    @property
    def effects(self):
        return self.cre.effects
    
    def optimize(self):
        self.cre.optimize()   
            
    def setValue(self, item, new_value):
        self.cre.setValue(item, new_value)
        
    def optimizeWeapon(self):
        self.cre.optimizeWeapon()
        
    def addSpell(self, spell):
        self.cre.addSpell(spell)
        
    def removeSpell(self, spell):
        self.cre.removeSpell(spell)
        
    def addItem(self, item):
        self.cre.addItem(item)
        
    def removeItem(self, item):
        self.cre.removeItem(item)
        
    def optimizeItem(self):
        self.cre.optimizeItem()
        
    def optimizeMem(self, num=100):
        self.cre.optimize(num)
        
    def memorize(self, spell, _num=1):
        self.cre.memorize(spell, _num)
        
    def memorize2(self, spells, forget=True):
        self.cre.memorize2(spells, forget)
        
    def forget(self, spell, _num=1):
        self.cre.forget(spell, _num)
        
    def forgetAll(self):
        self.cre.forgetAll()
        
    def pack(self, export=False):
        _cre_buffer = self.cre.pack()
        _buffer = self.buffer[self.header]
        _buffer.extend(_cre_buffer)
        _buffer.extend(self.buffer[self.tail])
        bio_buffer = self.zipfile.read(self.file_list[0])
        self.zipfile.close()
        with zipfile.ZipFile(self.filename, 'w') as target:
            for _f, _name in zip([bio_buffer, _buffer], self.file_list):
                target.writestr(_name, _f)
        if export == True:
            with open(self.file_list[1], 'wb') as target:
                target.write(_buffer)

In [9]:
class Save(BaseOps):
    
    def __init__(self, filename):
        self.filename = filename
        self.zipfile = zipfile.ZipFile(self.filename)
        self.filelist = self.zipfile.namelist()
        self.sav = bytearray(self.zipfile.read(self.filelist[0]))
        self.other_files = [self.zipfile.read(_) for _ in self.filelist[2:]]
        with open(filename, 'rb') as _source:
            self.buffer = bytearray(_source.read())
        self.header = 8
        self._data_length = 4
        self.gam_header_length = 180
        self.unpack()
        
    def _getFile(self, index):
        if isinstance(index, int):
            index = self.filelist[index]
        _info = self.zipfile.getinfo(index)
        _header_start = _info.header_offset
        _header = _info.FileHeader()
        _start = _header_start + len(_header)
        _end = _start + _info.compress_size
        return slice(_header_start, _start), slice(_start, _end), slice(_header_start, _end), slice(0, len(_header))
        
    def unzipSav(self):
        sav_buffer = self.sav[self.header:]
        while len(sav_buffer) > 0:
            _filename_length = self.bytes2Num(sav_buffer[:self._data_length])
            _pos = self._data_length
            _filename = str(sav_buffer[self._data_length:self._data_length + _filename_length], encoding='UTF-8')
            _pos += _filename_length
            _uncomp_size = self.bytes2Num(sav_buffer[_pos: _pos + self._data_length])
            _pos += self._data_length
            _comp_size = self.bytes2Num(sav_buffer[_pos: _pos + self._data_length])
            _pos += self._data_length
            _file = zlib.decompress(sav_buffer[_pos: _pos + _comp_size])
            _pos += _comp_size
            yield dict(filename=_filename, content=_file)
            sav_buffer = sav_buffer[_pos:]
            
    def unpack(self, export=False):
        _abs_count = 1
        _record_length = self.gam_header_length
        _info = self._getFile(1)
        self.head = self.buffer[:_info[2].start]
        self.tail = self.buffer[_info[2].stop:] 
        self.gam_buffer = self.buffer[_info[2]]
        if export:
            with open(self.filelist[1], 'wb') as _target:
                _target.write(self.gam_buffer)
        self.gam = GAM(self.filename, self.gam_buffer, _info[3])
            
        
    def pack(self):
        self.zipfile.close()
        _buffer = bytearray()
        _gam = self.gam.pack()
        _gam_content = _gam[self.gam.zipheader_length:]
        with zipfile.ZipFile(self.filename, 'w') as target:
            target.writestr(self.filelist[0], self.sav, compress_type=zipfile.ZIP_STORED)
            target.writestr(self.filelist[1], _gam_content, compress_type=zipfile.ZIP_STORED)
            for _f, _name in zip(self.other_files, self.filelist[2:]):
                target.writestr(_name, _f, compress_type=zipfile.ZIP_STORED)
            
    @property
    def CRE(self):
        return self.gam.PARTY
    
    @property
    def NPC(self):
        return self.gam.NPC
    
    @property
    def party(self):
        _data = pd.concat([_.data for _ in self.CRE], axis=1)
        return _data
    
    @property
    def npc(self):
        _data = pd.concat([_.data for _ in self.NPC], axis=1)
        return _data
    
    @property
    def globals(self):
        return self.gam.globals
    
    def getNPC(self, _npc):
        if isinstance(_npc, int) and _npc<len(self.NPC):
            return self.NPC[_npc]
        elif isinstance(_npc, str):
            for _char in self.NPC:
                if _char.name and _char.name[1:].lower() == _npc[1:].lower():
                    return _char
    
    def changeGlobal(self, _name, _value):
        self.gam.changeGlobal(_name, _value)
    
    def optimize(self, NPC=False):
        self.gam.optimize(NPC)
        
    def optimizeItem(self, NPC=False):
        self.gam.optimizeItem(NPC)
        
    def optimizeWeapon(self, NPC=False):
        self.gam.optimizeWeapon(NPC)
        
    def optimizeMem(self, NPC=False, num_party=100, num_npc=10):
        self.gam.optimizeMem(NPC, num_party, num_npc)

In [10]:
def optimize():
    filename = ['01FIGHT.bg1character','02THIEF.bg1character','03MAGE.bg1character','04PALAD.bg1character']
    a = [CHR(_) for _ in filename]
    a[0].setValue(Segments.Kit, Tables.Kit.GODHELM)
    a[2].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
    a[2].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
    a[3].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
    a[0].addSpell(Spells.Magic_CLERIC)
    a[1].addSpell(Spells.Magic_CLERIC)
    a[2].addSpell(Spells.Magic_WIZARD)
    a[3].addSpell(Spells.Magic_WIZARD)
    [_.optimize() for _ in a]
    [_.optimizeItem() for _ in a]
    [_.optimizeWeapon() for _ in a]
    [_.pack() for _ in a]

In [14]:
def optimize2():
    filename = r'GoodStart.bg1save'
    shutil.copyfile(filename, 'TestSave.bg1save')
    a = Save('TestSave.bg1save')
    a.CRE[0].setValue(Segments.Kit, Tables.Kit.GODHELM)
    a.CRE[2].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
    a.CRE[3].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
    a.CRE[2].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
    a.CRE[3].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
    a.optimize(NPC=True)
    a.optimizeItem(NPC=True)
    a.optimizeWeapon(NPC=True)
    a.optimizeMem()
    a.CRE[0].setValue(Segments.LvClass1, 40)
    a.CRE[0].setValue(Segments.LvClass2, 40)
    a.CRE[1].setValue(Segments.LvClass1, 40)
    a.CRE[1].setValue(Segments.LvClass2, 40)
    a.CRE[2].setValue(Segments.LvClass1, 40)
    a.CRE[2].setValue(Segments.LvClass2, 40)
    a.CRE[2].setValue(Segments.LvClass3, 40)
    a.CRE[3].setValue(Segments.LvClass1, 40)
    a.CRE[3].setValue(Segments.LvClass2, 40)
    a.CRE[3].setValue(Segments.LvClass3, 40)
    a.CRE[0].addSpell(Spells.CLERIC)
    a.CRE[1].addSpell(Spells.CLERIC)
    a.CRE[2].addSpell(Spells.WIZARD)
    a.CRE[3].addSpell(Spells.WIZARD)
    a.CRE[0].memorize2(CLERIC100, forget=True)
    a.CRE[1].memorize2(CLERIC100, forget=True)
    a.CRE[2].memorize2(WIZARD100, forget=True)
    a.CRE[3].memorize2(WIZARD100, forget=True)
    a.pack()
    return a

In [12]:
def optimize3():
    filename = r'GoodStart2.bg1save'
    shutil.copyfile(filename, 'TestSave.bg1save')
    a = Save('TestSave.bg1save')
    a.CRE[0].setValue(Segments.Class, Tables.Class.CLERIC_RANGER)
    a.CRE[1].setValue(Segments.Class, Tables.Class.FIGHTER_MAGE_THIEF)
    a.CRE[0].setValue(Segments.Kit, Tables.Kit.GODHELM)
    a.CRE[1].setValue(Segments.Kit, Tables.Kit.SHADOWDANCER)
    a.optimize(NPC=True)
    a.optimizeItem(NPC=True)
    a.optimizeWeapon(NPC=True)
    a.optimizeMem()
    a.CRE[0].setValue(Segments.LvClass1, 40)
    a.CRE[0].setValue(Segments.LvClass2, 40)
    a.CRE[1].setValue(Segments.LvClass1, 40)
    a.CRE[1].setValue(Segments.LvClass2, 40)
    a.CRE[1].setValue(Segments.LvClass3, 40)
    a.CRE[0].addSpell(Spells.CLERIC)
    a.CRE[1].addSpell(Spells.WIZARD)
    a.CRE[0].memorize2(CLERIC100, forget=True)
    a.CRE[1].memorize2(WIZARD100, forget=True)
    a.pack()
    return a

In [15]:
a = optimize2()

In [3]:
df = pd.read_csv('/Users/CIDER/工作/Python程序/游戏/博德之门/New/resources/BGEE/SPELL.csv', index_col=0)

In [4]:
df

,Class,Level,SpellCode,Spell
0,PALADIN,LEVEL_2,SPCL211,PALADIN_LAY_ON_HANDS
1,PALADIN,LEVEL_2,SPCL212,PALADIN_DETECT_EVIL
2,PALADIN,LEVEL_2,SPCL213,PALADIN_PROTECTION_FROM_EVIL
3,PALADIN,LEVEL_9,SPCL923,PALADIN_SUMMON_DEVA
4,SHAMAN,LEVEL_9,SPCL940,SHAMAN_SPIRIT_FORM
...,...,...,...,...
607,BARD,LEVEL_9,SPCL920,BARD_ENHANCED_BARD_SONG
608,BARD,LEVEL_9,SPCL921,BARD_MAGIC_FLUTE
609,MAGE,LEVEL_9,SPCL928,MAGE_EXTRA_LEVEL_6_SPELL
610,MAGE,LEVEL_9,SPCL929,MAGE_EXTRA_LEVEL_7_SPELL


In [21]:
df1 = pd.DataFrame([(_[0].value, _[1]) for _ in CLERIC100])

In [17]:
s1 = '''07/01/2022 07/01/2022 07/01/2022 07/01/2022 07/01/2022 07/01/2022 07/01/2022 07/01/2022 10/01/2022 10/01/2022 10/01/2022 10/01/2022 10/01/2022
07/05/2022 07/05/2022 07/05/2022 07/05/2022 11/23/2022 11/23/2022 11/23/2022 11/23/2022 10/03/2022 10/03/2022 10/03/2022 10/03/2022 11/22/2022
$63.73 $1,816.04 $3,256.35 $6,575.32 $1,584.09 $5,686.85 $7,917.21 $3,130.96 $819.13 $5,788.55 $2,839.66 $1,583.66 $1,624.31'''
s2 = '''07/01/2021 07/01/2021 07/01/2021 10/01/2021 10/01/2021 01/01/2022 01/01/2022 01/01/2022 04/01/2022 04/01/2022 04/01/2022 04/01/2022 04/01/2022 07/01/2022
02/28/2022 02/28/2022 02/28/2022 02/28/2022 05/27/2022 01/03/2022 01/03/2022 01/03/2022 04/04/2022 04/04/2022 04/04/2022 11/07/2022 11/07/2022 07/05/2022
$11,827.77 $11,768.19 $11,224.48 $2,539.58 $5,684.34 $9,737.33 $2,758.90 $19,799.23 $7,231.17 $16,447.38 $2,126.81 $5,849.80 $1,671.37 $939.32'''
s3 = '''07/01/2020 04/01/2021 04/01/2021
11/07/2022 08/08/2022 08/10/2022
$5,477.70 $4,850.88 $3,361.56'''
s = pd.concat([pd.DataFrame([_.split(' ') for _ in _s.split('\n')], index=['Buy', 'Sell', 'Amount']).T for _s in [s1, s2, s3]]).reset_index(drop=True)
s.Buy = pd.to_datetime(s.Buy)
s.Sell = pd.to_datetime(s.Sell)

In [23]:
s['Delta'] = s.Sell.apply(lambda x: abs(s.Buy - x).min())

In [25]:
s.sort_values('Delta')

,Buy,Sell,Amount,Delta
8,2022-10-01,2022-10-03,$819.13,2 days
9,2022-10-01,2022-10-03,"$5,788.55",2 days
10,2022-10-01,2022-10-03,"$2,839.66",2 days
11,2022-10-01,2022-10-03,"$1,583.66",2 days
20,2022-01-01,2022-01-03,"$19,799.23",2 days
19,2022-01-01,2022-01-03,"$2,758.90",2 days
18,2022-01-01,2022-01-03,"$9,737.33",2 days
23,2022-04-01,2022-04-04,"$2,126.81",3 days
22,2022-04-01,2022-04-04,"$16,447.38",3 days
21,2022-04-01,2022-04-04,"$7,231.17",3 days
